# EDA of Stop and Frisk Data
The goal of this notebook is to analyze stop and frisk data that is publicily available by the NYPD in aims to contribute to current events and work being done.

# Packages

In [1]:
# Set Up
import pandas as pd
import numpy as np

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Graphing
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,10)
import seaborn as sns
import plotly.express as px

# Loading and Cleaning Data
Our data comes from the NYPD's publicaly available stop, question, and frisk incidents. For now we will only look at 2019.

Source: https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page

In [2]:
#!pip install xlrd
raw = pd.read_excel('../data/sqf-2019.xlsx')

Becuase there are 83 features and over 13000 records, I'm going to break this dataset into smaller dataframes, clean seperately, then re-aggregate via the unique identifier key 'STOP_ID_ANONY'.

* location: describes the location of the SQF
* officer: describes the title, appearance, and behaviors of the officer
* suspect: describes the physical characteristics of the suspect
* sentiment: contains descriptions of the suspect from the perspective of the officer, useful for sentiment analysis

In [3]:
location = raw[['STOP_ID_ANONY', 'STOP_FRISK_DATE', 'STOP_FRISK_TIME',
                'YEAR2', 'MONTH2', 'DAY2','LOCATION_IN_OUT_CODE', 'JURISDICTION_CODE', 
                'JURISDICTION_DESCRIPTION', 'STOP_LOCATION_PRECINCT', 'STOP_LOCATION_SECTOR_CODE'
                ,'STOP_LOCATION_APARTMENT','STOP_LOCATION_FULL_ADDRESS','STOP_LOCATION_STREET_NAME'
                ,'STOP_LOCATION_X','STOP_LOCATION_Y','STOP_LOCATION_ZIP_CODE',
                'STOP_LOCATION_PATROL_BORO_NAME','STOP_LOCATION_BORO_NAME']]
location.head()

,STOP_ID_ANONY,STOP_FRISK_DATE,STOP_FRISK_TIME,YEAR2,MONTH2,DAY2,LOCATION_IN_OUT_CODE,JURISDICTION_CODE,JURISDICTION_DESCRIPTION,STOP_LOCATION_PRECINCT,STOP_LOCATION_SECTOR_CODE,STOP_LOCATION_APARTMENT,STOP_LOCATION_FULL_ADDRESS,STOP_LOCATION_STREET_NAME,STOP_LOCATION_X,STOP_LOCATION_Y,STOP_LOCATION_ZIP_CODE,STOP_LOCATION_PATROL_BORO_NAME,STOP_LOCATION_BORO_NAME
0,1,2019-01-02,14:30:00,2019,January,Wednesday,I,P,PSB,1,C,(null),230 VESEY STREET,VESEY STREET,979667,199737,(null),PBMS,MANHATTAN
1,2,2019-01-08,02:30:00,2019,January,Tuesday,O,P,PSB,1,C,(null),9 WHITE STREET,WHITE STREET,982650,201326,(null),PBMS,MANHATTAN
2,3,2019-01-12,16:54:00,2019,January,Saturday,I,P,PSB,1,D,(null),131 SPRING STREET,SPRING STREET,984063,203033,(null),PBMS,MANHATTAN
3,4,2019-01-14,21:21:00,2019,January,Monday,O,P,PSB,1,(,(null),GRAND STREET && 6TH AVE,GRAND STREET,982848,202677,(null),PBMS,MANHATTAN
4,5,2019-01-15,18:50:00,2019,January,Tuesday,O,P,PSB,1,D,(null),32 THOMPSON STREET,THOMPSON STREET,983100,202705,(null),PBMS,MANHATTAN


In [4]:
officer = raw[['STOP_ID_ANONY', 'ISSUING_OFFICER_RANK', 'ISSUING_OFFICER_COMMAND_CODE','SUPERVISING_OFFICER_RANK'
               ,'SUPERVISING_OFFICER_COMMAND_CODE','OFFICER_IN_UNIFORM_FLAG','ID_CARD_IDENTIFIES_OFFICER_FLAG'
               ,'SHIELD_IDENTIFIES_OFFICER_FLAG','VERBAL_IDENTIFIES_OFFICER_FLAG']]
officer.head()

,STOP_ID_ANONY,ISSUING_OFFICER_RANK,ISSUING_OFFICER_COMMAND_CODE,SUPERVISING_OFFICER_RANK,SUPERVISING_OFFICER_COMMAND_CODE,OFFICER_IN_UNIFORM_FLAG,ID_CARD_IDENTIFIES_OFFICER_FLAG,SHIELD_IDENTIFIES_OFFICER_FLAG,VERBAL_IDENTIFIES_OFFICER_FLAG
0,1,POM,1,SGT,1,Y,(null),(null),(null)
1,2,POM,1,SGT,1,N,(null),S,V
2,3,POM,1,SGT,1,Y,(null),(null),(null)
3,4,POM,1,SGT,1,Y,(null),(null),(null)
4,5,POM,1,SGT,1,N,(null),S,V


In [5]:
suspect = raw[['STOP_ID_ANONY','SUSPECTED_CRIME_DESCRIPTION', 'SUSPECT_ARRESTED_FLAG',
              'FRISKED_FLAG','SEARCHED_FLAG','ASK_FOR_CONSENT_FLG'
               ,'CONSENT_GIVEN_FLG','OTHER_CONTRABAND_FLAG','FIREARM_FLAG'
               ,'KNIFE_CUTTER_FLAG','OTHER_WEAPON_FLAG','WEAPON_FOUND_FLAG']]
suspect.head()

,STOP_ID_ANONY,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,FRISKED_FLAG,SEARCHED_FLAG,ASK_FOR_CONSENT_FLG,CONSENT_GIVEN_FLG,OTHER_CONTRABAND_FLAG,FIREARM_FLAG,KNIFE_CUTTER_FLAG,OTHER_WEAPON_FLAG,WEAPON_FOUND_FLAG
0,1,PETIT LARCENY,N,Y,N,Y,Y,N,(null),(null),(null),N
1,2,GRAND LARCENY,Y,N,Y,N,N,N,(null),(null),(null),N
2,3,PETIT LARCENY,Y,N,N,N,(,N,(null),(null),(null),N
3,4,ROBBERY,N,N,N,N,N,N,(null),(null),(null),N
4,5,ASSAULT,Y,Y,N,N,N,N,(null),(null),(null),N


In [6]:
sentiment = raw[['STOP_ID_ANONY', 'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION', 
                 'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION',
                 'DEMEANOR_OF_PERSON_STOPPED', 'SUSPECT_OTHER_DESCRIPTION']]
sentiment.head()

,STOP_ID_ANONY,OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION,OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION,DEMEANOR_OF_PERSON_STOPPED,SUSPECT_OTHER_DESCRIPTION
0,1,(null),(null),UNDERSTANDING,UNKNOWN
1,2,(null),(null),NORMAL,(null)
2,3,(null),(null),CALM,"BLACK LONG COAT, BLACK PANTS, BLACK FEDORA HAT..."
3,4,(null),(null),CONFUSED,(null)
4,5,(null),(null),NaN,(null)


### Let's begin with inspecting 
1. missing values
2. the datatype of each column
3. the possible values of each feature
4. the possible ways in which the feature can be utilized

This is way actually better done in Tableau.

The following code cell prints out the intital datatype assigned to each column by pandas import data function. Use at your own discretion 

In [7]:
# # Checking Datatype
# types = []
# for i in sqf2019.columns:
#     print(i, type(sqf2019[i][0]))

After inspecting the features via Tableau and the provided metadata, lets make the following changes:
1. drop RECORD_STATUS_CODE
2. change dummy variables with column name '%%FLAG' encoding from Y/N ==> 0/1
3.